# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Zhen Qian
#### Student ID: s3888611

Date: 03/10/2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy
* nltk
* itertools


## Introduction
Basic text processing.


## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import os
import re
from nltk.tokenize import RegexpTokenizer
from itertools import chain
from nltk.probability import *
from nltk.util import ngrams

### 1.1 Examining and loading data
- xamine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.


In [2]:
# Code to inspect the provided data file...
## examine the files and folders in root directory
rtdir = [f for f in os.listdir() if not f.startswith('.')]
print('root directory has', len(rtdir), 'files/folders')
print('they are:', rtdir)

root directory has 37 files/folders
they are: ['fast_text_weighted_projected.npy', 'count_vectors.npy', 'fast_text_weighted.npy', 'FT_desc.model', 'fast_text_unweighted.npy', 'task1.ipynb', 'glove_weighted.npy', 'task3.py', 'google_news_unweighted_projected.npy', 'ft_title_weighted.npy', 'glove_weighted_projected.npy', 'count_vectors_title.npy', 'google_title_weighted.npy', 'fast_text_unweighted_projected.npy', 'ft_title_unweighted.npy', 'test.py', 'glove_unweighted_projected.npy', 'test2.py', 'bigram.txt', 'google_title_unweighted.npy', 'classification.ipynb', 'glove', 'job_ads.txt', 'vocab_title.txt', 'google_news_weighted.npy', 'task2_3.ipynb', 'vocab.txt', 'job_categories.npy', 'glove_unweighted.npy', 'google_news_unweighted.npy', 'FT_desc.model.wv.vectors_ngrams.npy', 'count_vectors.txt', 'stopwords_en.txt', 'glove_title_weighted.npy', 'google_news_weighted_projected.npy', 'data', 'glove_title_unweighted.npy']


In [3]:
## examine files and folders in the data folder
datadir = [f for f in os.listdir('./data') if not f.startswith('.')]
print('data folder has', len(datadir), 'files/folders')
print('they are:', datadir)

data folder has 8 files/folders
they are: ['Sales', 'Accounting_Finance', 'IT', 'Healthcare_Nursing', 'Teaching', 'PR_Advertising_Marketing', 'Hospitality_Catering', 'Engineering']


In [4]:
## examine files in each ad folder
### number of files in each folder
ads_filenames = {}
for folder in datadir:
    ads_filenames[folder] = sorted([f for f in os.listdir('./data/'+folder) if f.endswith('.txt')])
for key, value in ads_filenames.items():
    print(key,'-', len(value), 'files')

Sales - 5349 files
Accounting_Finance - 7407 files
IT - 14353 files
Healthcare_Nursing - 8808 files
Teaching - 3779 files
PR_Advertising_Marketing - 2755 files
Hospitality_Catering - 4788 files
Engineering - 8210 files


In [5]:
#define lists to strore ids, categories, webindex, title, and description
ads_ids = []
ads_categories = []
ads_webinds = []
ads_titles = []
ads_descs = []

for folder, filenames in ads_filenames.items():
    for filename in filenames:
        
        ads_ids.append(re.findall('\d{5}', filename)[0])
        ads_categories.append(folder)
        
        with open('./data/'+folder+'/'+filename, 'r') as f:
            ads_text = f.read()
        
        ads_text = ads_text.splitlines()
        ads_webinds = ads_webinds+[line.strip().lstrip('Webindex:').strip() for line in ads_text if line.startswith('Webindex')]
        ads_titles = ads_titles+[line.strip().lstrip('Title:').strip() for line in ads_text if line.startswith('Title')]
        ads_descs = ads_descs+[line.strip().lstrip('Description:').strip() for line in ads_text if line.startswith('Description')]

In [6]:
# check if any files are missed
print(len(ads_ids))
print(len(ads_categories))
print(len(ads_webinds))
print(len(ads_titles))
print(len(ads_descs))

55449
55449
55449
55449
55449


### 1.2 Pre-processing data
Perform the required text pre-processing steps.

...... Sections and code blocks on basic text pre-processing


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [8]:
# define a function to check the processed results
def tokens_stats (list_name):
    ads_num = len(list_name)
    words_num = len(list(chain.from_iterable(list_name)))
    vocabs_num = len(set(chain.from_iterable(list_name)))
    ads_length = [len(ad) for ad in list_name]
    length_max = max(ads_length)
    length_min = min(ads_length)
    
    print('number of ads:', ads_num)
    print('number of words:', words_num)
    print('number of vocabularies:', vocabs_num)
    print('maximum length of an ad:', length_max)
    print('minimum length of an ad:', length_min)

In [9]:
# tokenization
tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
ads_descs_tokens = [tokenizer.tokenize(desc) for desc in ads_descs]

In [10]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 13799127
number of vocabularies: 112580
maximum length of an ad: 2001
minimum length of an ad: 10


In [11]:
# covert words to lower case
ads_descs_tokens = [[token.lower() for token in ad] for ad in ads_descs_tokens]

In [12]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 13799127
number of vocabularies: 89591
maximum length of an ad: 2001
minimum length of an ad: 10


In [13]:
# remove words with length less than 2
ads_descs_tokens = [[token for token in ad if len(token)>1] for ad in ads_descs_tokens]

In [14]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 13342925
number of vocabularies: 89565
maximum length of an ad: 1919
minimum length of an ad: 10


In [15]:
# remove stopwords
with open('stopwords_en.txt', 'r') as f:
    stopwords = f.read()

stopwords = stopwords.splitlines()
ads_descs_tokens = [[token for token in ad if token not in stopwords] for ad in ads_descs_tokens]

In [16]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 7863307
number of vocabularies: 89052
maximum length of an ad: 1132
minimum length of an ad: 7


In [17]:
# remove word only appear once base on term frequency
words = list(chain.from_iterable(ads_descs_tokens))
term_fd = FreqDist(words)
less_frequent_words = set(term_fd.hapaxes())
ads_descs_tokens = [[token for token in ad if token not in less_frequent_words] for ad in ads_descs_tokens]

In [18]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 7814343
number of vocabularies: 40088
maximum length of an ad: 1121
minimum length of an ad: 7


In [19]:
# remove top 50 most frequent words base on document frequency
ads_descs_tokens_docfd = [list(set(ad)) for ad in ads_descs_tokens]
words = list(chain.from_iterable(ads_descs_tokens_docfd))
doc_fd = FreqDist(words)
doc_fd.most_common(50)

[('experience', 43644),
 ('role', 34680),
 ('work', 33684),
 ('team', 32585),
 ('working', 30714),
 ('skills', 30412),
 ('client', 26899),
 ('job', 25552),
 ('business', 24739),
 ('uk', 24133),
 ('excellent', 22982),
 ('opportunity', 22678),
 ('company', 22263),
 ('management', 20620),
 ('required', 20555),
 ('development', 20223),
 ('apply', 20133),
 ('based', 19333),
 ('successful', 19118),
 ('join', 18682),
 ('www', 18421),
 ('salary', 18402),
 ('cv', 18383),
 ('support', 18286),
 ('knowledge', 17844),
 ('strong', 16475),
 ('environment', 16408),
 ('posted', 16398),
 ('jobseeking', 16342),
 ('candidate', 16304),
 ('originally', 16294),
 ('leading', 16194),
 ('high', 15922),
 ('service', 15623),
 ('manager', 15587),
 ('good', 15252),
 ('ability', 15154),
 ('including', 14857),
 ('position', 14564),
 ('services', 14501),
 ('benefits', 14434),
 ('training', 14218),
 ('essential', 13915),
 ('experienced', 13826),
 ('key', 13567),
 ('contact', 13551),
 ('level', 13523),
 ('recruitment', 

In [20]:
most_common_50 = [word[0] for word in doc_fd.most_common(50)]
ads_descs_tokens = [[token for token in ad if token not in most_common_50] for ad in ads_descs_tokens]

In [21]:
tokens_stats(ads_descs_tokens)

number of ads: 55449
number of words: 6239169
number of vocabularies: 40038
maximum length of an ad: 990
minimum length of an ad: 4


In [22]:
# Extract the top 10 Biggrams based on term frequency
words = list(chain.from_iterable(ads_descs_tokens))
bigrams = ngrams(words,2)
bigrams_fd = FreqDist(bigrams)
bigrams_fd.most_common(10)

[(('employment', 'agency'), 8055),
 (('track', 'record'), 5472),
 (('acting', 'employment'), 5095),
 (('sql', 'server'), 4804),
 (('asp', 'net'), 4687),
 (('relation', 'vacancy'), 3977),
 (('sales', 'executive'), 3619),
 (('chef', 'de'), 3586),
 (('nursing', 'home'), 3503),
 (('de', 'partie'), 3396)]

In [23]:
# define a dictionarty to store the bigrams
bigrams_dict = {}

for bigram in bigrams_fd.most_common(10):
    key = bigram[0][0]+' '+bigram[0][1]
    value = bigram[1]
    bigrams_dict[key] = value

bigrams_dict

{'employment agency': 8055,
 'track record': 5472,
 'acting employment': 5095,
 'sql server': 4804,
 'asp net': 4687,
 'relation vacancy': 3977,
 'sales executive': 3619,
 'chef de': 3586,
 'nursing home': 3503,
 'de partie': 3396}

## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- vocab.txt
- bigram.txt
- job_ads.txt

In [24]:
# save bigrams to txt file
bigrams_text_list = [key + ','+str(value) for key, value in bigrams_dict.items()]
bigrams_text = '\n'.join(bigrams_text_list)
    
with open('bigram.txt', 'w') as f:
    f.write(bigrams_text)

In [25]:
# save all job ads
job_ads_text_list = ['ID: ' + ads_ids[i] + '\n'
                    + 'Category: ' + ads_categories[i] + '\n'
                    + 'Webindex: '+ ads_webinds[i] + '\n'
                    + 'Title: ' + ads_titles[i] + '\n'
                    + 'Description: ' + ' '.join(ads_descs_tokens[i])
                    for i in range(len(ads_ids))]

job_ads_text = '\n'.join(job_ads_text_list)
    
with open('job_ads.txt', 'w') as f:
    f.write(job_ads_text)

In [26]:
# save the vocabulories
vocabs = set(chain.from_iterable(ads_descs_tokens))
vocabs = sorted(vocabs)

vocabs_text_list = [vocabs[i] + ':' + str(i) for i in range(len(vocabs))]
vocabs_text = '\n'.join(vocabs_text_list)

with open('vocab.txt', 'w') as f:
    f.write(vocabs_text)

## Summary
Basic text processing was done.